# Temporal Taylor Diagrams

In [259]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [260]:
import os

import dask
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import regionmask
import xarray as xr
import xesmf as xe
from dask.distributed import Client
from evaltools import obs
from evaltools.obs import eobs_mapping
from evaltools.utils import short_iid
from tools import (
    TaylorDiagram,
    check_equal_period,
    create_cordex_grid,
    fix_360_longitudes,
    height_temperature_correction,
    load_obs,
    mask_invalid,
    open_datasets,
    regional_mean,
    regional_means,
    regrid_dsets,
    select_season,
    standardize_unit,
    var_dic,
    variable_mapping,
)

dask.config.set(scheduler="single-threaded")

In [261]:
client = Client(dashboard_address="localhost:8000", threads_per_worker=1)
client

/mnt/CORDEX_CMIP6_tmp/user_tmp/jdiezsierra/conda_envs/evaltools/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8000 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37311 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:37311/status,
Dashboard: http://127.0.0.1:37311/status,Workers: 16
Total threads: 16,Total memory: 125.79 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40433,Workers: 0
Dashboard: http://127.0.0.1:37311/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:44537,Total threads: 1
Dashboard: http://127.0.0.1:44059/status,Memory: 7.86 GiB
Nanny: tcp://127.0.0.1:41281,


In [262]:
def compute_tcoiav(model_ds, reference_ds):
    """
    Compute the Temporal Correlation of Interannual Variability (TCOIAV) between model and reference data.

    The Temporal Correlation of Interannual Variability (TCOIAV) measures the correlation between the
    interannual variability of spatially averaged annual or seasonal mean values of the model and reference
    datasets for a selected subregion. It quantifies how well the model captures the year-to-year variations
    observed in the reference data.

    A higher positive TCOIAV value indicates that the model accurately captures the interannual variability
    observed in the reference data, while a lower or negative TCOIAV value indicates that the model's
    interannual variability deviates significantly from that of the reference data.

    Parameters:
    model_ds (xarray.DataArray): The model data with spatial dimensions.
    reference_ds (xarray.DataArray): The reference data with spatial dimensions.

    Returns:
    float: The TCOIAV value.
    """
    # Compute the annual or seasonal mean values
    model_mean = model_ds.groupby("time.year").mean("time")
    reference_mean = reference_ds.groupby("time.year").mean("time")

    if "lon" in reference_mean.coords:
        # Spatially average these mean values over the subregion
        model_mean = model_mean.mean(dim=["lat", "lon"])
        reference_mean = reference_mean.mean(dim=["lat", "lon"])

    # Compute the temporal correlation of interannual variability
    tcoiav = xr.corr(model_mean, reference_mean, dim="year")

    return tcoiav


# RIAV: ratio (model over reference) of temporal standard
# deviations of interannual time series of spatially aver-
# aged annual or seasonal mean values of a selected sub-
# region.


def compute_riav(model_ds, reference_ds):
    """
    Compute the Ratio of Interannual Variability (RIAV) between model and reference data.

    The Ratio of Interannual Variability (RIAV) is the ratio of the temporal standard deviations
    of interannual time series of spatially averaged annual or seasonal mean values between the model
    and reference datasets for a selected subregion. It quantifies the relative temporal variability
    in the model data compared to the reference data.

    An RIAV value greater than 1 indicates that the model data has higher temporal variability
    than the reference data, while an RIAV value less than 1 indicates lower temporal variability
    in the model data compared to the reference data.

    Parameters:
    model_ds (xarray.DataArray): The model data with spatial dimensions.
    reference_ds (xarray.DataArray): The reference data with spatial dimensions.

    Returns:
    float: The RIAV value.
    """
    # Compute the annual or seasonal mean values
    model_mean = model_ds.groupby("time.year").mean("time")
    reference_mean = reference_ds.groupby("time.year").mean("time")

    if "lon" in reference_mean.coords:
        # Spatially average these mean values over the subregion
        model_mean = model_mean.mean(dim=["lat", "lon"])
        reference_mean = reference_mean.mean(dim=["lat", "lon"])

    # Compute the temporal standard deviations of the interannual time series
    model_std = model_mean.std(dim="year")
    reference_std = reference_mean.std(dim="year")

    # Compute the ratio of these standard deviations (RIAV)
    riav = model_std / reference_std

    return riav

In [311]:
# Parameter papermill
index = "pr"
frequency = "mon"
domain = "EUR-11"
regridding = "bilinear"
period = slice("1989", "2008")
period = slice("1991", "2020")

reference_regions = "PRUDENCE"
parent = False

In [312]:
save_results_path = os.path.abspath(
    os.path.join(os.getcwd(), "..", "intermediate-results")
)
save_figure_path = os.path.abspath(os.path.join(os.getcwd(), "..", "plots"))

In [313]:
variable = var_dic[index]["variable"]

In [314]:
eur_colors = pd.read_csv("eurocordex_models.csv")

In [315]:
# prudence
regions = regionmask.defined_regions.prudence

In [316]:
rotated_grid = create_cordex_grid("EUR-11")  # No matter CMIP5 or CMIP6

## E-OBS is used as the reference dataset for all the analysis
It is used to calculate bias not only respect to CORDEX, but also in comparison wit other reanalyses and observational dataset, to assess the uncertaintly of the observational dataset

In [317]:
# load, regrid and calculate seasonal means
eobs_var = [key for key, value in eobs_mapping.items() if value == variable][0]
eobs = obs.eobs(variables=eobs_var, add_mask=False).sel(time=period)
eobs = mask_invalid(eobs, vars=eobs_var, threshold=0.1)
eobs = standardize_unit(eobs, variable)
# eobs = load_eobs(add_mask=False, to_cf=False, variable = variable)
# unmapped_to_nan, see https://github.com/pangeo-data/xESMF/issues/56
regridder = xe.Regridder(eobs, rotated_grid, method=regridding, unmapped_to_nan=True)
ref_on_rotated = regridder(eobs)
if not check_equal_period(ref_on_rotated, period):
    print(f"Temporal coverage of dataset does not match with {period}")
ref_regions = regional_mean(
    ref_on_rotated[eobs_var], regions, aggr=var_dic[index]["aggr"]
)
ref_regions_seasons = select_season(ref_regions).compute()

/mnt/CORDEX_CMIP6_tmp/user_tmp/jdiezsierra/conda_envs/evaltools/lib/python3.10/site-packages/distributed/client.py:3363: UserWarning: Sending large graph of size 25.48 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


## CERRA and ERA5

In [318]:
dsets = {}
for dset in var_dic[variable]["datasets"]:
    ds = load_obs(variable, dset, add_fx=True, mask=True)
    ds = ds.sel(time=period).compute()
    ds = fix_360_longitudes(ds, lonname="longitude")
    if not variable_mapping[dset][variable] == variable:
        ds = ds.rename_vars({variable_mapping[dset][variable]: variable})
    ds = standardize_unit(ds, variable)
    dsets[dset] = ds

merging era5 with orog
merging era5 with sftlf
Convert precipitation from meters to millimeters (mm).


/mnt/CORDEX_CMIP6_tmp/user_tmp/jdiezsierra/conda_envs/evaltools/lib/python3.10/site-packages/distributed/client.py:3363: UserWarning: Sending large graph of size 9.89 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/mnt/CORDEX_CMIP6_tmp/user_tmp/jdiezsierra/conda_envs/evaltools/lib/python3.10/site-packages/distributed/client.py:3363: UserWarning: Sending large graph of size 9.89 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


merging cerra-land with orog
merging cerra-land with sftlf


/mnt/CORDEX_CMIP6_tmp/user_tmp/jdiezsierra/conda_envs/evaltools/lib/python3.10/site-packages/distributed/client.py:3363: UserWarning: Sending large graph of size 18.74 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Convert precipitation from meters to millimeters (mm).


In [319]:
for dset in dsets.keys():
    if not check_equal_period(dsets[dset], period):
        print(f"Temporal coverage of {dset} does not match with {period}")

In [320]:
for dset, ds in dsets.items():
    regridder = xe.Regridder(ds, rotated_grid, method=regridding, unmapped_to_nan=True)
    dsets[dset] = regridder(ds)

In [321]:
if variable == "tas":
    for dset in dsets:
        h_c = height_temperature_correction(dsets[dset].orog, ref_on_rotated.elevation)
        dsets[dset]["tas"] = dsets[dset].tas - h_c.fillna(0)

In [322]:
obs_regions = regional_means(dsets, regions, aggr=var_dic[index]["aggr"]).compute()

In [323]:
obs_regions_seasons = select_season(obs_regions).compute()

In [324]:
model_ds = obs_regions_seasons.copy()
reference_ds = ref_regions_seasons.copy()

In [325]:
diffs = {}
for dset_id in obs_regions_seasons.iid:
    dset_id = str(dset_id.values)
    model_id = obs_regions_seasons[variable].sel(iid=dset_id)
    diffs[dset_id] = compute_tcoiav(model_id, ref_regions_seasons).compute()

obs_tcoiav = xr.concat(
    list(diffs.values()),
    dim=xr.DataArray(
        list(map(lambda x: x, diffs.keys())),
        dims="dset_id",
    ),
    compat="override",
    coords="minimal",
)

diffs = {}
for dset_id in obs_regions_seasons.iid:
    dset_id = str(dset_id.values)
    model_id = obs_regions_seasons[variable].sel(iid=dset_id)
    diffs[dset_id] = compute_riav(model_id, ref_regions_seasons).compute()

obs_riav = xr.concat(
    list(diffs.values()),
    dim=xr.DataArray(
        list(map(lambda x: x, diffs.keys())),
        dims="dset_id",
    ),
    compat="override",
    coords="minimal",
)

## CMIP6

In [326]:
mip_era = "CMIP6"
driving_source_id = "ERA5"
# Define how to merge the files in xarray

In [327]:
dsets = open_datasets(
    [variable],
    frequency=frequency,
    driving_source_id=driving_source_id,
    mask=True,
    add_missing_bounds=False,
)

Found: ['ALARO1-SFX', 'CCLM6-0-1-URB', 'CCLM6-0-1-URB-ESG', 'CNRM-ALADIN64E1', 'HCLIM43-ALADIN', 'ICON-CLM-202407-1-1', 'RACMO23E', 'REMO2020-2-2', 'REMO2020-2-2-MR2', 'REMO2020-2-2-iMOVE', 'REMO2020-2-2-iMOVE-LUC', 'RegCM5-0'] for variables: ['pr']

--> The keys in the returned dictionary of datasets are constructed as follows:
	'project_id.domain_id.institution_id.driving_source_id.driving_experiment_id.driving_variant_label.source_id.version_realization.frequency.version'


decoding dataset CORDEX-CMIP6.EUR-12.KNMI.ERA5.evaluation.r1i1p1f1.RACMO23E.v1-r1.fx.v20241216
Found 62 datasets
decoding dataset CORDEX-CMIP6.EUR-12.GERICS.ERA5.evaluation.r1i1p1f1.REMO2020-2-2.v1-r1.fx.v20241120
Warning for CORDEX-CMIP6.EUR-12.GERICS.ERA5.evaluation.r1i1p1f1.REMO2020-2-2.v1-r1.fx.v20241120: Variable(s) referenced in cell_measures not in variables: ['areacella']
Warning for CORDEX-CMIP6.EUR-12.GERICS.ERA5.evaluation.r1i1p1f1.REMO2020-2-2.v1-r1.fx.v20241120: Variable(s) referenced in cell_measures not in variables: ['areacella']
Found 62 datasets
decoding dataset CORDEX-CMIP6.EUR-12.KNMI.EC-Earth3.historical.r1i1p1f1.RACMO23E.v1-r1.fx.v20241216
Found 62 datasets
decoding dataset CORDEX-CMIP6.EUR-12.HCLIMcom-SMHI.EC-Earth3-Veg.ssp126.r3i1p1f1.HCLIM43-ALADIN.v1-r1.fx.v20241205
Found 62 datasets
decoding dataset CORDEX-CMIP6.EUR-12.CLMcom-Hereon.ERA5.evaluation.r1i1p1f1.ICON-CLM-202407-1-1.v1-r1.fx.v20240920
Warning for CORDEX-CMIP6.EUR-12.CLMcom-Hereon.ERA5.evaluation.r1

/mnt/CORDEX_CMIP6_tmp/user_tmp/jdiezsierra/evaltools/evaltools/fix.py:88: UserWarning: Grid mapping has (198.0, 39.25) which is inconsistent with (-162.0, 39.25) for EUR-12 and CORDEX-CMIP6.EUR-12.ICTP.ERA5.evaluation.r1i1p1f1.RegCM5-0.v1-r1.fx.v20250415.
  warnings.warn(message)


Fix failed for CORDEX-CMIP6.EUR-12.ICTP.ERA5.evaluation.r1i1p1f1.RegCM5-0.v1-r1.fx.v20250415: Grid mapping has (198.0, 39.25) which is inconsistent with (-162.0, 39.25) for EUR-12 and CORDEX-CMIP6.EUR-12.ICTP.ERA5.evaluation.r1i1p1f1.RegCM5-0.v1-r1.fx.v20250415.
Dataset CORDEX-CMIP6.EUR-12.ICTP.ERA5.evaluation.r1i1p1f1.RegCM5-0.v1-r1.fx.v20250415 will be ignored...


/mnt/CORDEX_CMIP6_tmp/user_tmp/jdiezsierra/evaltools/evaltools/fix.py:88: UserWarning: Grid mapping has (198.0, 39.25) which is inconsistent with (-162.0, 39.25) for EUR-12 and CORDEX-CMIP6.EUR-12.ICTP.ERA5.evaluation.r1i1p1f1.RegCM5-0.v1-r1.mon.v20250415.
  warnings.warn(message)
/mnt/CORDEX_CMIP6_tmp/user_tmp/jdiezsierra/evaltools/evaltools/fix.py:69: UserWarning: Grid mapping name latitude_longitude is not supported for CORDEX-CMIP6.EUR-12.CNRM-MF.CNRM-ESM2-1.historical.r1i1p1f2.CNRM-ALADIN64E1.v1-r1.fx.v20250328
  warnings.warn(


Fix failed for CORDEX-CMIP6.EUR-12.ICTP.ERA5.evaluation.r1i1p1f1.RegCM5-0.v1-r1.mon.v20250415: Grid mapping has (198.0, 39.25) which is inconsistent with (-162.0, 39.25) for EUR-12 and CORDEX-CMIP6.EUR-12.ICTP.ERA5.evaluation.r1i1p1f1.RegCM5-0.v1-r1.mon.v20250415.
Dataset CORDEX-CMIP6.EUR-12.ICTP.ERA5.evaluation.r1i1p1f1.RegCM5-0.v1-r1.mon.v20250415 will be ignored...
Fix failed for CORDEX-CMIP6.EUR-12.CNRM-MF.CNRM-ESM2-1.historical.r1i1p1f2.CNRM-ALADIN64E1.v1-r1.fx.v20250328: Grid mapping name latitude_longitude is not supported for CORDEX-CMIP6.EUR-12.CNRM-MF.CNRM-ESM2-1.historical.r1i1p1f2.CNRM-ALADIN64E1.v1-r1.fx.v20250328
Dataset CORDEX-CMIP6.EUR-12.CNRM-MF.CNRM-ESM2-1.historical.r1i1p1f2.CNRM-ALADIN64E1.v1-r1.fx.v20250328 will be ignored...
Warning for CORDEX-CMIP6.EUR-12.CLMcom-KUL.ERA5.evaluation.r1i1p1f1.CCLM6-0-1-URB-ESG.v1-r1.fx.v20250409: "Receive multiple variables for key 'grid_mapping': ['crs', 'rotated_pole']. Expected only one. Please pass a list ['grid_mapping'] inste

/mnt/CORDEX_CMIP6_tmp/user_tmp/jdiezsierra/evaltools/evaltools/fix.py:69: UserWarning: Grid mapping name latitude_longitude is not supported for CORDEX-CMIP6.EUR-12.CNRM-MF.CNRM-ESM2-1.ssp370.r1i1p1f2.CNRM-ALADIN64E1.v1-r1.fx.v20250328
  warnings.warn(


Fix failed for CORDEX-CMIP6.EUR-12.CNRM-MF.CNRM-ESM2-1.ssp370.r1i1p1f2.CNRM-ALADIN64E1.v1-r1.fx.v20250328: Grid mapping name latitude_longitude is not supported for CORDEX-CMIP6.EUR-12.CNRM-MF.CNRM-ESM2-1.ssp370.r1i1p1f2.CNRM-ALADIN64E1.v1-r1.fx.v20250328
Dataset CORDEX-CMIP6.EUR-12.CNRM-MF.CNRM-ESM2-1.ssp370.r1i1p1f2.CNRM-ALADIN64E1.v1-r1.fx.v20250328 will be ignored...
merging CORDEX-CMIP6.EUR-12.CLMcom-CMCC.ERA5.evaluation.r1i1p1f1.CCLM6-0-1-URB.v1-r1.mon.v20250201 with CORDEX-CMIP6.EUR-12.CLMcom-CMCC.ERA5.evaluation.r1i1p1f1.CCLM6-0-1-URB.v1-r1.fx.v20250201
merging CORDEX-CMIP6.EUR-12.GERICS.ERA5.evaluation.r1i1p1f1.REMO2020-2-2-iMOVE-LUC.v1-r1.mon.v20250515 with CORDEX-CMIP6.EUR-12.GERICS.ERA5.evaluation.r1i1p1f1.REMO2020-2-2-iMOVE-LUC.v1-r1.fx.v20250515
merging CORDEX-CMIP6.EUR-12.GERICS.ERA5.evaluation.r1i1p1f1.REMO2020-2-2-MR2.v1-r1.mon.v20241120 with CORDEX-CMIP6.EUR-12.GERICS.ERA5.evaluation.r1i1p1f1.REMO2020-2-2-MR2.v1-r1.fx.v20241120
merging CORDEX-CMIP6.EUR-12.GERICS.ERA5.

In [328]:
for dset in dsets.keys():
    dsets[dset] = dsets[dset].sel(time=period).compute()

In [329]:
for dset in dsets.keys():
    if not check_equal_period(dsets[dset], period):
        print(f"Temporal coverage of {dset} does not match with {period}")

Temporal coverage of CORDEX-CMIP6.EUR-12.RMIB-UGent.ERA5.evaluation.r1i1p1f1.ALARO1-SFX.v1-r1.mon.v20241009 does not match with slice('1991', '2020', None)


In [330]:
for dset in dsets.keys():
    dsets[dset] = standardize_unit(dsets[dset], variable)

Convert precipitation from kg/m/s² to mm/s.
Convert precipitation from meters to millimeters (mm).
Convert precipitation from kg/m/s² to mm/s.
Convert precipitation from meters to millimeters (mm).
Convert precipitation from kg/m/s² to mm/s.
Convert precipitation from meters to millimeters (mm).
Convert precipitation from kg/m/s² to mm/s.
Convert precipitation from meters to millimeters (mm).
Convert precipitation from kg/m/s² to mm/s.
Convert precipitation from meters to millimeters (mm).
Convert precipitation from kg/m/s² to mm/s.
Convert precipitation from meters to millimeters (mm).
Convert precipitation from kg/m/s² to mm/s.
Convert precipitation from meters to millimeters (mm).
Convert precipitation from kg/m/s² to mm/s.
Convert precipitation from meters to millimeters (mm).
Convert precipitation from kg/m/s² to mm/s.
Convert precipitation from meters to millimeters (mm).
Convert precipitation from kg/m/s² to mm/s.
Convert precipitation from meters to millimeters (mm).
Convert pr

In [331]:
rotated_grid = create_cordex_grid(domain)
dsets = regrid_dsets(dsets, rotated_grid, method=regridding)

regridding CORDEX-CMIP6.EUR-12.RMIB-UGent.ERA5.evaluation.r1i1p1f1.ALARO1-SFX.v1-r1.mon.v20241009 with grid_mapping: lambert_conformal_conic
xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            bilinear_483x483_412x424.nc 
Reuse pre-computed weights? False 
Input grid shape:           (483, 483) 
Output grid shape:          (412, 424) 
Periodic in longitude?      False
regridding CORDEX-CMIP6.EUR-12.CNRM-MF.ERA5.evaluation.r1i1p1f1.CNRM-ALADIN64E1.v1-r1.mon.v20250505 with grid_mapping: lambert_conformal_conic
xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            bilinear_453x453_412x424.nc 
Reuse pre-computed weights? False 
Input grid shape:           (453, 453) 
Output grid shape:          (412, 424) 
Periodic in longitude?      False
regridding CORDEX-CMIP6.EUR-12.HCLIMcom-SMHI.ERA5.evaluation.r1i1p1f1.HCLIM43-ALADIN.v1-r1.mon.v20241205 with grid_mapping: lambert_conformal_conic
xESMF Regridder 
Regridding algorithm:       b

In [332]:
if variable == "tas":
    for dset in dsets:
        h_c = height_temperature_correction(dsets[dset].orog, ref_on_rotated.elevation)
        dsets[dset]["tas"] = dsets[dset].tas - h_c.fillna(0)

In [333]:
dset_id_regions = regional_means(dsets, regions, aggr=var_dic[index]["aggr"]).compute()

In [334]:
dset_id_regions_seasons = select_season(dset_id_regions)

In [335]:
diffs = {}
for dset_id in dset_id_regions_seasons.iid:
    print(str(dset_id.values))
    dset_id = str(dset_id.values)
    model_id = dset_id_regions_seasons[variable].sel(iid=dset_id)
    diffs[dset_id] = compute_tcoiav(model_id, ref_regions_seasons).compute()

dset_id_tcoiav = xr.concat(
    list(diffs.values()),
    dim=xr.DataArray(
        list(map(lambda x: short_iid(x, ["source_id"]), diffs.keys())),
        dims="dset_id",
    ),
    compat="override",
    coords="minimal",
)

CORDEX-CMIP6.EUR-12.CLMcom-CMCC.ERA5.evaluation.r1i1p1f1.CCLM6-0-1-URB.v1-r1.mon.v20250201
CORDEX-CMIP6.EUR-12.GERICS.ERA5.evaluation.r1i1p1f1.REMO2020-2-2-iMOVE-LUC.v1-r1.mon.v20250515
CORDEX-CMIP6.EUR-12.GERICS.ERA5.evaluation.r1i1p1f1.REMO2020-2-2-MR2.v1-r1.mon.v20241120
CORDEX-CMIP6.EUR-12.GERICS.ERA5.evaluation.r1i1p1f1.REMO2020-2-2.v1-r1.mon.v20241120
CORDEX-CMIP6.EUR-12.RMIB-UGent.ERA5.evaluation.r1i1p1f1.ALARO1-SFX.v1-r1.mon.v20241009
CORDEX-CMIP6.EUR-12.KNMI.ERA5.evaluation.r1i1p1f1.RACMO23E.v1-r1.mon.v20241216
CORDEX-CMIP6.EUR-12.CLMcom-Hereon.ERA5.evaluation.r1i1p1f1.ICON-CLM-202407-1-1.v1-r1.mon.v20240920
CORDEX-CMIP6.EUR-12.CLMcom-KUL.ERA5.evaluation.r1i1p1f1.CCLM6-0-1-URB-ESG.v1-r1.mon.v20250409
CORDEX-CMIP6.EUR-12.CNRM-MF.ERA5.evaluation.r1i1p1f1.CNRM-ALADIN64E1.v1-r1.mon.v20250505
CORDEX-CMIP6.EUR-12.HCLIMcom-SMHI.ERA5.evaluation.r1i1p1f1.HCLIM43-ALADIN.v1-r1.mon.v20241205
CORDEX-CMIP6.EUR-12.GERICS.ERA5.evaluation.r1i1p1f1.REMO2020-2-2-iMOVE.v1-r1.mon.v20250515


In [336]:
diffs = {}
for dset_id in dset_id_regions_seasons.iid:
    dset_id = str(dset_id.values)
    print(dset_id)
    model_id = dset_id_regions_seasons[variable].sel(iid=dset_id)
    diffs[dset_id] = compute_riav(model_id, ref_regions_seasons).compute()

dset_id_riav = xr.concat(
    list(diffs.values()),
    dim=xr.DataArray(
        list(map(lambda x: short_iid(x, ["source_id"]), diffs.keys())),
        dims="dset_id",
    ),
    compat="override",
    coords="minimal",
)

CORDEX-CMIP6.EUR-12.CLMcom-CMCC.ERA5.evaluation.r1i1p1f1.CCLM6-0-1-URB.v1-r1.mon.v20250201
CORDEX-CMIP6.EUR-12.GERICS.ERA5.evaluation.r1i1p1f1.REMO2020-2-2-iMOVE-LUC.v1-r1.mon.v20250515
CORDEX-CMIP6.EUR-12.GERICS.ERA5.evaluation.r1i1p1f1.REMO2020-2-2-MR2.v1-r1.mon.v20241120
CORDEX-CMIP6.EUR-12.GERICS.ERA5.evaluation.r1i1p1f1.REMO2020-2-2.v1-r1.mon.v20241120
CORDEX-CMIP6.EUR-12.RMIB-UGent.ERA5.evaluation.r1i1p1f1.ALARO1-SFX.v1-r1.mon.v20241009
CORDEX-CMIP6.EUR-12.KNMI.ERA5.evaluation.r1i1p1f1.RACMO23E.v1-r1.mon.v20241216
CORDEX-CMIP6.EUR-12.CLMcom-Hereon.ERA5.evaluation.r1i1p1f1.ICON-CLM-202407-1-1.v1-r1.mon.v20240920
CORDEX-CMIP6.EUR-12.CLMcom-KUL.ERA5.evaluation.r1i1p1f1.CCLM6-0-1-URB-ESG.v1-r1.mon.v20250409
CORDEX-CMIP6.EUR-12.CNRM-MF.ERA5.evaluation.r1i1p1f1.CNRM-ALADIN64E1.v1-r1.mon.v20250505
CORDEX-CMIP6.EUR-12.HCLIMcom-SMHI.ERA5.evaluation.r1i1p1f1.HCLIM43-ALADIN.v1-r1.mon.v20241205
CORDEX-CMIP6.EUR-12.GERICS.ERA5.evaluation.r1i1p1f1.REMO2020-2-2-iMOVE.v1-r1.mon.v20250515


In [337]:
dset_id_tcoiav.to_netcdf(
    f"{save_results_path}/{variable}_CMIP6_tcoiav_{period.start}-{period.stop}.nc"
)
dset_id_riav.to_netcdf(
    f"{save_results_path}/{variable}_CMIP6_riav_{period.start}-{period.stop}.nc"
)

## CMIP5

In [338]:
mip_era = "CMIP5"
driving_source_id = "ERAINT"

In [339]:
dsets = open_datasets(
    [variable],
    frequency=frequency,
    driving_source_id=driving_source_id,
    mask=True,
    add_missing_bounds=False,
)

Found: ['ALADIN53', 'ALADIN63', 'ALARO-0', 'CCLM4-8-17', 'COSMO-crCLIM-v1-1', 'HadREM3-GA7-05', 'RACMO22E', 'RCA4', 'REMO2009', 'REMO2015', 'RegCM4-2'] for variables: ['pr']

--> The keys in the returned dictionary of datasets are constructed as follows:
	'project_id.domain_id.institution_id.driving_source_id.driving_experiment_id.driving_variant_label.source_id.version_realization.frequency.version'


decoding dataset CORDEX.EUR-11.CLMcom-ETH.ERAINT.evaluation.r0i0p0.COSMO-crCLIM-v1-1.v1.fx.v20191210
Found 22 datasets
decoding dataset CORDEX.EUR-11.RMIB-UGent.ERAINT.evaluation.r1i1p1.ALARO-0.v1.fx.v20170523
Found 22 datasets
decoding dataset CORDEX.EUR-11.SMHI.ERAINT.evaluation.r1i1p1.RCA4.v1.mon.v20131026
Found 22 datasets
decoding dataset CORDEX.EUR-11.KNMI.ERAINT.evaluation.r1i1p1.RACMO22E.v1.mon.v20140218
Found 22 datasets
decoding dataset CORDEX.EUR-11.RMIB-UGent.ERAINT.evaluation.r1i1p1.ALARO-0.v1.mon.v20170207
Found 22 datasets
decoding dataset CORDEX.EUR-11.DHMZ.ERAINT.evaluation.r1i1p1.RegCM4-2.v1.mon.v20150527
Found 22 datasets
decoding dataset CORDEX.EUR-11.CNRM.ERAINT.evaluation.r1i1p1.ALADIN53.v1.mon.v20150127
Found 22 datasets
decoding dataset CORDEX.EUR-11.CLMcom.ERAINT.evaluation.r0i0p0.CCLM4-8-17.v1.fx.v20140515
Warning for CORDEX.EUR-11.CLMcom.ERAINT.evaluation.r0i0p0.CCLM4-8-17.v1.fx.v20140515: Variable(s) referenced in cell_measures not in variables: ['areacella'

In [340]:
for dset in dsets.keys():
    dsets[dset] = dsets[dset].sel(time=period).compute()

In [341]:
for dset in dsets.keys():
    if not check_equal_period(dsets[dset], period):
        print(f"Temporal coverage of {dset} does not match with {period}")

Temporal coverage of CORDEX.EUR-11.SMHI.ERAINT.evaluation.r1i1p1.RCA4.v1.mon.v20131026 does not match with slice('1991', '2020', None)
Temporal coverage of CORDEX.EUR-11.KNMI.ERAINT.evaluation.r1i1p1.RACMO22E.v1.mon.v20140218 does not match with slice('1991', '2020', None)
Temporal coverage of CORDEX.EUR-11.RMIB-UGent.ERAINT.evaluation.r1i1p1.ALARO-0.v1.mon.v20170207 does not match with slice('1991', '2020', None)
Temporal coverage of CORDEX.EUR-11.DHMZ.ERAINT.evaluation.r1i1p1.RegCM4-2.v1.mon.v20150527 does not match with slice('1991', '2020', None)
Temporal coverage of CORDEX.EUR-11.CNRM.ERAINT.evaluation.r1i1p1.ALADIN53.v1.mon.v20150127 does not match with slice('1991', '2020', None)
Temporal coverage of CORDEX.EUR-11.GERICS.ERAINT.evaluation.r1i1p1.REMO2015.v1.mon.v20180813 does not match with slice('1991', '2020', None)
Temporal coverage of CORDEX.EUR-11.MOHC.ERAINT.evaluation.r1i1p1.HadREM3-GA7-05.v1.mon.v20200330 does not match with slice('1991', '2020', None)
Temporal coverage 

In [342]:
for dset in dsets.keys():
    dsets[dset] = standardize_unit(dsets[dset], variable)

Convert precipitation from kg/m/s² to mm/s.
Convert precipitation from meters to millimeters (mm).
Convert precipitation from kg/m/s² to mm/s.
Convert precipitation from meters to millimeters (mm).
Convert precipitation from kg/m/s² to mm/s.
Convert precipitation from meters to millimeters (mm).
Convert precipitation from kg/m/s² to mm/s.
Convert precipitation from meters to millimeters (mm).
Convert precipitation from kg/m/s² to mm/s.
Convert precipitation from meters to millimeters (mm).
Convert precipitation from kg/m/s² to mm/s.
Convert precipitation from meters to millimeters (mm).
Convert precipitation from kg/m/s² to mm/s.
Convert precipitation from meters to millimeters (mm).
Convert precipitation from kg/m/s² to mm/s.
Convert precipitation from meters to millimeters (mm).
Convert precipitation from kg/m/s² to mm/s.
Convert precipitation from meters to millimeters (mm).
Convert precipitation from kg/m/s² to mm/s.
Convert precipitation from meters to millimeters (mm).
Convert pr

In [343]:
rotated_grid = create_cordex_grid(domain)
dsets = regrid_dsets(dsets, rotated_grid, method=regridding)

regridding CORDEX.EUR-11.RMIB-UGent.ERAINT.evaluation.r1i1p1.ALARO-0.v1.mon.v20170207 with grid_mapping: lambert_conformal_conic
xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            bilinear_485x485_412x424.nc 
Reuse pre-computed weights? False 
Input grid shape:           (485, 485) 
Output grid shape:          (412, 424) 
Periodic in longitude?      False
regridding CORDEX.EUR-11.DHMZ.ERAINT.evaluation.r1i1p1.RegCM4-2.v1.mon.v20150527 with grid_mapping: lambert_conformal_conic
xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            bilinear_551x551_412x424.nc 
Reuse pre-computed weights? False 
Input grid shape:           (551, 551) 
Output grid shape:          (412, 424) 
Periodic in longitude?      False
regridding CORDEX.EUR-11.CNRM.ERAINT.evaluation.r1i1p1.ALADIN53.v1.mon.v20150127 with grid_mapping: lambert_conformal_conic
xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            bilinear_453x453_4

In [344]:
if variable == "tas":
    for dset in dsets:
        h_c = height_temperature_correction(dsets[dset].orog, ref_on_rotated.elevation)
        dsets[dset]["tas"] = dsets[dset].tas - h_c.fillna(0)

In [345]:
del dsets[
    "CORDEX.EUR-11.MPI-CSC.ERAINT.evaluation.r1i1p1.REMO2009.v1.mon.v20160525"
]  # problems with calendar (one year less)

In [346]:
dset_id_regions = regional_means(dsets, regions, aggr=var_dic[index]["aggr"]).compute()

In [347]:
dset_id_regions_seasons = select_season(dset_id_regions).compute()

In [348]:
diffs = {}
for dset_id in dset_id_regions_seasons.iid:
    dset_id = str(dset_id.values)
    model_id = dset_id_regions_seasons[variable].sel(iid=dset_id)
    diffs[dset_id] = compute_tcoiav(model_id, ref_regions_seasons).compute()

dset_id_tcoiav = xr.concat(
    list(diffs.values()),
    dim=xr.DataArray(
        list(map(lambda x: short_iid(x, ["source_id"]), diffs.keys())),
        dims="dset_id",
    ),
    compat="override",
    coords="minimal",
)

In [349]:
diffs = {}
for dset_id in dset_id_regions_seasons.iid:
    dset_id = str(dset_id.values)
    print(dset_id)
    model_id = dset_id_regions_seasons[variable].sel(iid=dset_id)
    diffs[dset_id] = compute_riav(model_id, ref_regions_seasons).compute()

dset_id_riav = xr.concat(
    list(diffs.values()),
    dim=xr.DataArray(
        list(map(lambda x: short_iid(x, ["source_id"]), diffs.keys())),
        dims="dset_id",
    ),
    compat="override",
    coords="minimal",
)

CORDEX.EUR-11.SMHI.ERAINT.evaluation.r1i1p1.RCA4.v1.mon.v20131026
CORDEX.EUR-11.KNMI.ERAINT.evaluation.r1i1p1.RACMO22E.v1.mon.v20140218
CORDEX.EUR-11.RMIB-UGent.ERAINT.evaluation.r1i1p1.ALARO-0.v1.mon.v20170207
CORDEX.EUR-11.DHMZ.ERAINT.evaluation.r1i1p1.RegCM4-2.v1.mon.v20150527
CORDEX.EUR-11.CNRM.ERAINT.evaluation.r1i1p1.ALADIN53.v1.mon.v20150127
CORDEX.EUR-11.GERICS.ERAINT.evaluation.r1i1p1.REMO2015.v1.mon.v20180813
CORDEX.EUR-11.MOHC.ERAINT.evaluation.r1i1p1.HadREM3-GA7-05.v1.mon.v20200330
CORDEX.EUR-11.CNRM.ERAINT.evaluation.r1i1p1.ALADIN63.v1.mon.v20191118
CORDEX.EUR-11.CLMcom.ERAINT.evaluation.r1i1p1.CCLM4-8-17.v1.mon.v20140515
CORDEX.EUR-11.CLMcom-ETH.ERAINT.evaluation.r1i1p1.COSMO-crCLIM-v1-1.v1.mon.v20191210


In [350]:
dset_id_tcoiav.to_netcdf(
    f"{save_results_path}/{variable}_CMIP5_tcoiav_{period.start}-{period.stop}.nc"
)
dset_id_riav.to_netcdf(
    f"{save_results_path}/{variable}_CMIP5_riav_{period.start}-{period.stop}.nc"
)

#### Load results for both CMIP5 and CMIP6 simulations

In [351]:
dset_id_tcoiav_CMIP6 = xr.open_dataset(
    f"{save_results_path}/{variable}_CMIP6_tcoiav_{period.start}-{period.stop}.nc"
)
dset_id_riav_CMIP6 = xr.open_dataset(
    f"{save_results_path}/{variable}_CMIP6_riav_{period.start}-{period.stop}.nc"
)

In [352]:
CMIP6_coord = xr.DataArray(
    np.full(len(dset_id_tcoiav_CMIP6["dset_id"]), "CMIP6"),
    dims="dset_id",
    coords={"dset_id": dset_id_tcoiav_CMIP6["dset_id"]},
    name="mip_era",
)
dset_id_tcoiav_CMIP6 = dset_id_tcoiav_CMIP6.assign_coords(mip_era=CMIP6_coord)
dset_id_riav_CMIP6 = dset_id_riav_CMIP6.assign_coords(mip_era=CMIP6_coord)

In [353]:
dset_id_tcoiav_CMIP5 = xr.open_dataset(
    f"{save_results_path}/{variable}_CMIP5_tcoiav_{period.start}-{period.stop}.nc"
)
dset_id_riav_CMIP5 = xr.open_dataset(
    f"{save_results_path}/{variable}_CMIP5_riav_{period.start}-{period.stop}.nc"
)

In [354]:
CMIP5_coord = xr.DataArray(
    np.full(len(dset_id_tcoiav_CMIP5["dset_id"]), "CMIP5"),
    dims="dset_id",
    coords={"dset_id": dset_id_tcoiav_CMIP5["dset_id"]},
    name="mip_era",
)
dset_id_tcoiav_CMIP5 = dset_id_tcoiav_CMIP5.assign_coords(mip_era=CMIP5_coord)
dset_id_riav_CMIP5 = dset_id_riav_CMIP5.assign_coords(mip_era=CMIP5_coord)

In [355]:
seasons_marker = {"winter": "o", "summer": "^"}

In [356]:
parent_str = "no-parent"

In [ ]:
from PIL import Image

# Reference std
stdref = 1

regions = ["EA", "IP", "ME", "SC"]

for n_r, region in enumerate(regions):
    fig = plt.figure()

    dia = TaylorDiagram(stdref, fig=fig, label="Reference")
    # dia.samplePoints[0].set_color('r')  # Mark reference point as a red star

    for season, mark in seasons_marker.items():

        if parent is True:
            parent_str = "parent"

            # cmip5
            rho = dset_id_tcoiav_CMIP5.isel(
                region=np.where(dset_id_tcoiav_CMIP5.abbrevs == region)[0],
                season=np.where(dset_id_tcoiav_CMIP5.season == season)[0],
            ).squeeze()
            std = dset_id_riav_CMIP5.isel(
                region=np.where(dset_id_riav_CMIP5.abbrevs == region)[0],
                season=np.where(dset_id_riav_CMIP5.season == season)[0],
            ).squeeze()
            # Add models to Taylor diagram
            for i, model in enumerate(dset_id_tcoiav_CMIP5.dset_id.data):
                mip_era = eur_colors["mip_era"][eur_colors["model"] == model].values
                color = eur_colors["color"][eur_colors["model"] == model].values[0]
                dia.add_sample(
                    std.sel(dset_id=model)[list(std.data_vars)[0]].item(),
                    rho.sel(dset_id=model)[list(rho.data_vars)[0]].item(),
                    marker=mark,
                    ms=5,
                    ls="",
                    mfc="none",
                    mec=color,
                    label=f"{model}_{season}",
                )

        # cmip6
        rho = dset_id_tcoiav_CMIP6.isel(
            region=np.where(dset_id_tcoiav_CMIP6.abbrevs == region)[0],
            season=np.where(dset_id_tcoiav_CMIP6.season == season)[0],
        ).squeeze()
        std = dset_id_riav_CMIP6.isel(
            region=np.where(dset_id_riav_CMIP6.abbrevs == region)[0],
            season=np.where(dset_id_riav_CMIP6.season == season)[0],
        ).squeeze()
        # Add models to Taylor diagram
        for i, model in enumerate(dset_id_tcoiav_CMIP6.dset_id.data):
            mip_era = eur_colors["mip_era"][eur_colors["model"] == model].values
            color = eur_colors["color"][eur_colors["model"] == model].values[0]
            dia.add_sample(
                std.sel(dset_id=model)[list(std.data_vars)[0]].item(),
                rho.sel(dset_id=model)[list(rho.data_vars)[0]].item(),
                marker=mark,
                ms=5,
                ls="",
                mfc=mcolors.to_rgba(color, 0.5),
                mec=color,
                label=f"{model}_{season}",
            )

        # obs
        rho = obs_tcoiav.isel(
            region=np.where(obs_tcoiav.abbrevs == region)[0],
            season=np.where(obs_tcoiav.season == season)[0],
        ).squeeze()
        std = obs_riav.isel(
            region=np.where(obs_riav.abbrevs == region)[0],
            season=np.where(obs_riav.season == season)[0],
        ).squeeze()
        # Add models to Taylor diagram
        for i, model in enumerate(obs_tcoiav.dset_id.data):
            if "era5" in model:
                color = "magenta"
            else:
                color = "black"

            dia.add_sample(
                std.sel(dset_id=model),
                rho.sel(dset_id=model),
                marker=mark,
                ms=8,
                ls="",
                mfc="none",
                mec=color,
                label=f"{model}_{season}",
            )

    # Add correlation lines
    dia.add_correlation_lines()

    # Add RMS contours, and label them
    contours = dia.add_contours(levels=2, colors="0.5")  # 5 levels in grey
    plt.clabel(contours, inline=1, fontsize=10, fmt="%.1f")

    if n_r == 1:
        # Add a figure legend and title
        fig.legend(
            dia.samplePoints,
            [p.get_label() for p in dia.samplePoints],
            numpoints=1,
            prop=dict(size=5),
            loc="upper right",
        )

    fig.text(0.25, 0.83, region, fontsize=12, fontweight="bold", va="top", ha="left")

    fig.savefig(
        f"taylor_{parent_str}_{region}_{period.start}-{period.stop}.png", dpi=300
    )
    plt.close(fig)

imgs = [
    Image.open(f"taylor_{parent_str}_{r}_{period.start}-{period.stop}.png")
    for r in regions
]
imgs = [img.crop(img.getbbox()) for img in imgs]

w, h = imgs[0].size

final_img = Image.new("RGB", (2 * w, 2 * h), "white")

final_img.paste(imgs[0], (0, 0))
final_img.paste(imgs[1], (w, 0))
final_img.paste(imgs[2], (0, h))
final_img.paste(imgs[3], (w, h))

final_img.save(
    f"{save_figure_path}/PRUDENCE_A_taylor_temporal_{variable}_{period.start}-{period.stop}.png"
)

/tmp/ipykernel_4059362/1717529038.py:82: MatplotlibDeprecationWarning: Passing `apply_theta_transforms=True` (the default) is deprecated since Matplotlib 3.9. Support for this will be removed in Matplotlib in 3.11. To prevent this warning, set `apply_theta_transforms=False`, and make sure to shift theta values before being passed to this transform.
  plt.clabel(contours, inline=1, fontsize=10, fmt='%.1f')
/tmp/ipykernel_4059362/1717529038.py:82: MatplotlibDeprecationWarning: Passing `apply_theta_transforms=True` (the default) is deprecated since Matplotlib 3.9. Support for this will be removed in Matplotlib in 3.11. To prevent this warning, set `apply_theta_transforms=False`, and make sure to shift theta values before being passed to this transform.
  plt.clabel(contours, inline=1, fontsize=10, fmt='%.1f')
/tmp/ipykernel_4059362/1717529038.py:82: MatplotlibDeprecationWarning: Passing `apply_theta_transforms=True` (the default) is deprecated since Matplotlib 3.9. Support for this will b

In [ ]:
from PIL import Image

# Reference std
stdref = 1

regions = ["AL", "BI", "FR", "MD"]

for n_r, region in enumerate(regions):
    fig = plt.figure()

    dia = TaylorDiagram(stdref, fig=fig, label="Reference")
    # dia.samplePoints[0].set_color('r')  # Mark reference point as a red star

    for season, mark in seasons_marker.items():

        if parent is True:
            parent_str = "parent"

            # cmip5
            rho = dset_id_tcoiav_CMIP5.isel(
                region=np.where(dset_id_tcoiav_CMIP5.abbrevs == region)[0],
                season=np.where(dset_id_tcoiav_CMIP5.season == season)[0],
            ).squeeze()
            std = dset_id_riav_CMIP5.isel(
                region=np.where(dset_id_riav_CMIP5.abbrevs == region)[0],
                season=np.where(dset_id_riav_CMIP5.season == season)[0],
            ).squeeze()
            # Add models to Taylor diagram
            for i, model in enumerate(dset_id_tcoiav_CMIP5.dset_id.data):
                mip_era = eur_colors["mip_era"][eur_colors["model"] == model].values
                color = eur_colors["color"][eur_colors["model"] == model].values[0]
                dia.add_sample(
                    std.sel(dset_id=model)[list(std.data_vars)[0]].item(),
                    rho.sel(dset_id=model)[list(rho.data_vars)[0]].item(),
                    marker=mark,
                    ms=5,
                    ls="",
                    mfc="none",
                    mec=color,
                    label=f"{model}_{season}",
                )

        # cmip6
        rho = dset_id_tcoiav_CMIP6.isel(
            region=np.where(dset_id_tcoiav_CMIP6.abbrevs == region)[0],
            season=np.where(dset_id_tcoiav_CMIP6.season == season)[0],
        ).squeeze()
        std = dset_id_riav_CMIP6.isel(
            region=np.where(dset_id_riav_CMIP6.abbrevs == region)[0],
            season=np.where(dset_id_riav_CMIP6.season == season)[0],
        ).squeeze()
        # Add models to Taylor diagram
        for i, model in enumerate(dset_id_tcoiav_CMIP6.dset_id.data):
            mip_era = eur_colors["mip_era"][eur_colors["model"] == model].values
            color = eur_colors["color"][eur_colors["model"] == model].values[0]
            dia.add_sample(
                std.sel(dset_id=model)[list(std.data_vars)[0]].item(),
                rho.sel(dset_id=model)[list(rho.data_vars)[0]].item(),
                marker=mark,
                ms=5,
                ls="",
                mfc=mcolors.to_rgba(color, 0.5),
                mec=color,
                label=f"{model}_{season}",
            )

        # obs
        rho = obs_tcoiav.isel(
            region=np.where(obs_tcoiav.abbrevs == region)[0],
            season=np.where(obs_tcoiav.season == season)[0],
        ).squeeze()
        std = obs_riav.isel(
            region=np.where(obs_riav.abbrevs == region)[0],
            season=np.where(obs_riav.season == season)[0],
        ).squeeze()
        # Add models to Taylor diagram
        for i, model in enumerate(obs_tcoiav.dset_id.data):
            if "era5" in model:
                color = "magenta"
            else:
                color = "black"

            dia.add_sample(
                std.sel(dset_id=model),
                rho.sel(dset_id=model),
                marker=mark,
                ms=8,
                ls="",
                mfc="none",
                mec=color,
                label=f"{model}_{season}",
            )

    # Add correlation lines
    dia.add_correlation_lines()

    # Add RMS contours, and label them
    contours = dia.add_contours(levels=2, colors="0.5")  # 5 levels in grey
    plt.clabel(contours, inline=1, fontsize=10, fmt="%.1f")

    if n_r == 1:
        # Add a figure legend and title
        fig.legend(
            dia.samplePoints,
            [p.get_label() for p in dia.samplePoints],
            numpoints=1,
            prop=dict(size=5),
            loc="upper right",
        )

    fig.text(0.25, 0.83, region, fontsize=12, fontweight="bold", va="top", ha="left")

    fig.savefig(
        f"taylor_{parent_str}_{region}_{period.start}-{period.stop}.png", dpi=300
    )
    plt.close(fig)

imgs = [
    Image.open(f"taylor_{parent_str}_{r}_{period.start}-{period.stop}.png")
    for r in regions
]
imgs = [img.crop(img.getbbox()) for img in imgs]

w, h = imgs[0].size

final_img = Image.new("RGB", (2 * w, 2 * h), "white")

final_img.paste(imgs[0], (0, 0))
final_img.paste(imgs[1], (w, 0))
final_img.paste(imgs[2], (0, h))
final_img.paste(imgs[3], (w, h))

final_img.save(
    f"{save_figure_path}/PRUDENCE_B_taylor_temporal_{variable}_{period.start}-{period.stop}.png"
)

/tmp/ipykernel_4059362/1570122679.py:82: MatplotlibDeprecationWarning: Passing `apply_theta_transforms=True` (the default) is deprecated since Matplotlib 3.9. Support for this will be removed in Matplotlib in 3.11. To prevent this warning, set `apply_theta_transforms=False`, and make sure to shift theta values before being passed to this transform.
  plt.clabel(contours, inline=1, fontsize=10, fmt='%.1f')
/tmp/ipykernel_4059362/1570122679.py:82: MatplotlibDeprecationWarning: Passing `apply_theta_transforms=True` (the default) is deprecated since Matplotlib 3.9. Support for this will be removed in Matplotlib in 3.11. To prevent this warning, set `apply_theta_transforms=False`, and make sure to shift theta values before being passed to this transform.
  plt.clabel(contours, inline=1, fontsize=10, fmt='%.1f')
/tmp/ipykernel_4059362/1570122679.py:82: MatplotlibDeprecationWarning: Passing `apply_theta_transforms=True` (the default) is deprecated since Matplotlib 3.9. Support for this will b